In [2]:
import numpy as np
total = [0]
def my_matmult(m1, m2):
    ans_mat = np.empty((0,m2.shape[1]))
    for row in m1:
        ans_row = np.array([])
        for col in m2.T:
            ans_row = np.append(ans_row, np.dot(row, col))
            total[0] += len(row)*2
        ans_mat = np.vstack((ans_mat,ans_row))
    return ans_mat

a = np.random.rand(3, 2)
b = np.random.rand(2, 3)

print(np.array_equal(a@b, my_matmult(a,b)))


True


In [3]:
def tensor_product(m1, m2):
  ans = np.empty( (0, m1.shape[-2], m2.shape[-1]) )
  for i in range(m1.shape[0]):
    matmult = np.expand_dims(my_matmult(m1[i], m2[i]), axis=0)
    ans = np.vstack((ans, matmult))
  return ans

a = np.random.rand(2, 3, 4)
b = np.random.rand(2, 4, 2)

print(np.allclose(tensor_product(a,b), a @ b))


True


In [4]:
def nd_tensor_product(m1, m2):
  ans_shape = m1.shape[:-2] + (m1.shape[-2], m2.shape[-1])
  ans = np.empty(ans_shape)
  for i in range(m1.shape[0]):
    if len(m1.shape) == 3:
        ans[i] = my_matmult(m1[i], m2[i])

    else:
        ans[i] = nd_tensor_product(m1[i], m2[i])
  return ans

a = np.random.rand(12, 4, 64)
b = np.random.rand(12, 64, 4)

total = [0]

print(np.allclose(nd_tensor_product(a,b), a @ b))
print(total)

True
[24576]


In [2]:
import numpy as np

In [18]:
def absmax_quantization(m1, type=np.int8):
    matrix = np.empty((0, m1.shape[-1]))
    row_constants = []
    data_max = np.iinfo(type).max
    for row in m1:
        norm_range = round((data_max/2)**(1/2))

        row_constant = norm_range/np.max(abs(row))
        tow_type = (row * row_constant).astype(type)
        matrix.vstack(tow_type)
        row_constants += row_constant
    return matrix, row_constants

# def zeropoint_quantization(m1, type=np.int8):
#     return (m1 * 2* np.iinfo(type).max)/ (max(m1) - min(m1))


In [25]:
def matix_absmax_quantization(m1, type=np.int8):
    data_max = np.iinfo(type).max
    norm_range = round((data_max/2)**(1/2))

    quantification_constant = norm_range/np.max(abs(m1))
    tow_type = (m1 * quantification_constant).astype(type)
    return tow_type, quantification_constant

In [26]:


def my_matmul(m1, m2):
    m1, m1_const = matix_absmax_quantization(m1, np.int64)
    m2, m2_const = matix_absmax_quantization(m2, np.int64)

    ans = np.matmul(m1,m2)

    ans = (ans.astype(np.float64))/ (m1_const * m2_const)
    return ans

def nd_tensor_product(m1, m2):
    ans_shape = m1.shape[:-2] + (m1.shape[-2], m2.shape[-1])
    ans = np.empty(ans_shape)
    if len(ans_shape) < 3:
        return my_matmul(m1, m2)
    else:
        for i in range(ans_shape[0]):
            ans[i] = nd_tensor_product(m1[i], m2[i])
    return ans

a = np.random.uniform(-100, 100, size=(2, 3))
b = np.random.uniform(-100, 100, size=(3, 5))


print(np.matmul(a,b))
nd_tensor_product(a,b)


[[  224.97050426  6879.94932527 12458.00860865  1289.62296289
  -6600.2008004 ]
 [-1802.48000164  5809.90714037 13136.94936619 -3306.20671234
  -4287.85140144]]


array([[   224.97050576,   6879.94932521,  12458.00860189,
          1289.62296178,  -6600.2007987 ],
       [ -1802.4800004 ,   5809.90713925, -11932.71954306,
         -3306.20671205,  -4287.85140358]])